In [1]:
import numpy as np
from numba import jit, prange
from dask import delayed, compute

## Fast boostrap resampling when there are multiple datasets (of same size) to boostrap

In [2]:
def multi_bootstrap(data, boots):
    """
    Keyword arguments:
    data -- numpy multi-dimentional array 
    boot -- number of bootstraps  
    
    """
    designs = data.shape[0]
    samples = data.shape[1]
    
    to_return = np.empty((designs, boots))
    
    for design in range(designs):
        
        to_return[design:design+1] = bootstrap(data[design], boots)
        
    return to_return

In [3]:
@jit(nopython=False)
def multi_bootstrap_jit(data, boots):
    """
    Keyword arguments:
    data -- numpy multi-dimentional array 
    boot -- number of bootstraps  
    
    """
    designs = data.shape[0]
    samples = data.shape[1]
    
    to_return = np.empty((designs, boots))
    
    for design in range(designs):
        
        to_return[design:design+1] = bootstrap(data[design], boots)
        
    return to_return

In [5]:
@jit(nopython=False)
def multi_bootstrap_par(data, boots):
    """
    Keyword arguments:
    data -- numpy multi-dimentional array 
    boot -- number of bootstraps  
    
    """
    designs = data.shape[0]
    samples = data.shape[1]
    
    to_return = np.empty((designs, boots))
    
    for design in range(designs):
        
        to_return[design:design+1] = bootstrap_par(data[design], boots)
        
    return to_return

In [25]:
@jit(nopython=True)
def bootstrap(data, boots):
    """
    Create bootstrap datasets that represent the distribution of the mean.
    Returns a numpy array containing the bootstrap datasets 
    
    Keyword arguments:
    data -- numpy array of systems to boostrap
    boots -- number of bootstrap (default = 1000)
    """
    
    bs_data = np.empty(boots)
    
    for b in range(boots):
        
        total=0
        
        for s in range(data.shape[0]):
        
            total += data[round(np.random.uniform(0, data.shape[0]-1))]

        bs_data[b] = total / data.shape[0]

    return bs_data

In [8]:
@jit(nopython=True, parallel=True)
def bootstrap_par(data, boots):
    """
    Create bootstrap datasets that represent the distribution of the mean.
    Returns a numpy array containing the bootstrap datasets 
    
    Keyword arguments:
    data -- numpy array of systems to boostrap
    boots -- number of bootstrap (default = 1000)
    """
    
    bs_data = np.empty(boots)
    
    for b in prange(boots):
        
        total=0
        
        for s in range(data.shape[0]):
        
            total += data[round(np.random.uniform(0, data.shape[0]-1))]

        bs_data[b] = total / data.shape[0]

    return bs_data

## Small Problem 10 initial datasets

In [21]:
sample = np.arange(100).reshape(10, 10)
sample.shape

(10, 10)

In [22]:
%%timeit
x = multi_bootstrap(sample, 1000)

1.05 ms ± 48.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%%timeit
x = multi_bootstrap_jit(sample, 1000)

923 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%%timeit
x = multi_bootstrap_par(sample, 1000)

773 µs ± 3.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


multi-core processing improves performance even with a problem as small as 10 initial datasets

## Larger Problem.  1000 initial datasets

In [9]:
sample = np.arange(10000).reshape(1000, 10)
sample.shape

(1000, 10)

NumPy Implementation

In [10]:
%%timeit
x = multi_bootstrap(sample, 1000)

96.5 ms ± 1.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


NumPy and Numba JIT implementation

In [20]:
%%timeit
x = multi_bootstrap_jit(sample, 1000)

93 ms ± 774 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


NumPy, JIT and Parallel (multi-core) Implementation

In [14]:
%%timeit
x = multi_bootstrap_par(sample, 1000)

77.5 ms ± 114 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
